In [1]:
# kernel auto reload modules when the underlying code is chaneged, instead of having to reset the runtime.
%load_ext autoreload
%autoreload 2

from copy import deepcopy
import os
from tqdm import tqdm
from glob import glob

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision
import torchvision.transforms as ttf
from torch.cuda.amp import GradScaler, autocast
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score, roc_auc_score
import hydra
from omegaconf import OmegaConf
import wandb

from models.convnext import convnext_t, my_convnext
from datasets.classification import ClassificationTestSet
from datasets.verification import VerificationDataset
from datasets.transform import AlbumTransforms, train_transforms, val_transforms
from utils.utils import weight_decay_custom, compute_kl_loss, SAM
from run import train, test, inference, face_embedding, verification_inference, gen_cls_submission, gen_ver_submission

In [23]:
# load classification dataset / loader
BASE_DIR = '/shared/youngkim/hw2p2'
CLS_DIR = os.path.join(BASE_DIR, '11-785-s22-hw2p2-classification')
VER_DIR = os.path.join(BASE_DIR, '11-785-s22-hw2p2-verification')

CLS_TRAIN_DIR = os.path.join(CLS_DIR, "classification/classification/train") # This is a smaller subset of the data. Should change this to classification/classification/train
CLS_VAL_DIR = os.path.join(CLS_DIR, "classification/classification/dev")
CLS_TEST_DIR = os.path.join(CLS_DIR, "classification/classification/test")


face_norm_mean = (0.511, 0.402, 0.351)
face_norm_std = (0.270, 0.235, 0.222)

test_transforms = A.Compose(
    [
        # A.CenterCrop(height=200, width=200),
        A.Resize(224, 224),
        A.Normalize(
                mean=face_norm_mean, 
                std=face_norm_std, 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2(),
    ]
)

tta_transforms = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.1, 
                           rotate_limit=15, 
                           p=0.5),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.OneOf([
            A.HueSaturationValue(
                hue_shift_limit=10, 
                sat_shift_limit=15, 
                val_shift_limit=10, 
                p=0.5
            ),
            A.RGBShift(
                r_shift_limit=13,
                g_shift_limit=10,
                b_shift_limit=8,
                p=0.5
            )
        ], p=1),  
        A.Normalize(
                mean=face_norm_mean, 
                std=face_norm_std, 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()
    ]
)
val_dataset = torchvision.datasets.ImageFolder(CLS_VAL_DIR,
                                                transform=AlbumTransforms(tta_transforms))
test_dataset = ClassificationTestSet(CLS_TEST_DIR, AlbumTransforms(tta_transforms))

valid_loader = DataLoader(val_dataset, batch_size=256, shuffle=False,
                        drop_last=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False,
                        drop_last=False, num_workers=1)

In [3]:
device = torch.device('cuda:2' if torch.cuda.is_available() else "cpu")
save_name = 'convnext-4772-16:13:41:23'
model = my_convnext(dropout=0, block_nums=[4,7,7,2]).to(device)

In [4]:
weights_path = '/shared/youngkim/hw2p2/weights/'
pretrained_path = os.path.join(weights_path, f'{save_name}.pth')
checkpoint = torch.load(pretrained_path)
model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# start_epoch = checkpoint['epoch'] + 1
print(f"Model loaded: {pretrained_path}")

Model loaded: /shared/youngkim/hw2p2/weights/convnext-4772-16:13:41:23.pth


In [24]:
def tta(model, device, num_epochs, test_loader, valid_loader=None):
    model.eval()

    if valid_loader is not None:
        val_cls_out = torch.zeros([35000,7000], device=device, dtype=torch.float32)
        true_y_list = []
        for epoch in range(num_epochs):
            i = 0
            for data, true_y in tqdm(valid_loader, total=len(valid_loader), desc='Valid', position=0, leave=True):
                data = data.to(device)
                true_y = true_y.to(device)                
                
                with torch.no_grad():   
                    output = model(data)
                    if i == 0:
                        val_out = deepcopy(output)
                        i += 1
                    else:
                        val_out = torch.concat((val_out, output), dim=0)

                    if epoch == 0:
                        true_y_list.extend(true_y.tolist())
        
            val_cls_out = val_cls_out + val_out

        val_pred_y = torch.argmax(val_cls_out, axis=1)
        val_accuracy =  accuracy_score(true_y_list, val_pred_y.tolist())

        print(val_accuracy)

    test_cls_out = torch.zeros([35000,7000], device=device, dtype=torch.float32)
    for epoch in range(num_epochs):
        i = 0
        for data in tqdm(test_loader, total=len(test_loader), desc='Test', position=0, leave=True):
            data = data.to(device)
            
            with torch.no_grad():   
                output = model(data)
                if i == 0:
                    test_out = deepcopy(output)
                    i += 1
                else:
                    test_out = torch.concat((test_out, output), dim=0)
    
        test_cls_out = test_cls_out + test_out

    pred_y = torch.argmax(test_cls_out, axis=1).tolist()

    return pred_y

# tta_pred_5epochs = tta(model, device, 5, test_loader, valid_loader)

In [20]:
base_line = tta(model, device, 1, test_loader, valid_loader)

Valid: 100%|██████████| 137/137 [00:52<00:00,  2.60it/s]

0.9078571428571428


In [17]:
center_crop = tta(model, device, 1, test_loader, valid_loader)

Valid: 100%|██████████| 137/137 [00:55<00:00,  2.47it/s]

0.9066857142857143


In [25]:
tta_pred_10epochs = tta(model, device, 10, test_loader, valid_loader)

Valid:   0%|          | 0/137 [00:00<?, ?it/s]

In [8]:
submissions_path = '/shared/youngkim/hw2p2/submissions/'

def gen_submission(predictions):
    test_names = [str(i).zfill(6) + ".jpg" for i in range(len(predictions))]
    submission = pd.DataFrame(zip(test_names, predictions), columns=['id', 'label'])
    submission.to_csv(os.path.join(submissions_path, f'tta-5.csv'), index=False)

In [9]:
gen_submission(tta_pred_5epochs)

In [14]:
submission_name = 'tta-5'
sub_dir = '/shared/youngkim/hw2p2/submissions'
cls_csv = os.path.join(sub_dir, f'{submission_name}.csv')
!kaggle competitions submit -c 11-785-s22-hw2p2-classification -f {cls_csv} -m {submission_name}

100%|█████████████████████████████████████████| 541k/541k [00:03<00:00, 160kB/s]
Successfully submitted to Face Recognition